In [63]:
from llama_index import SimpleDirectoryReader
import arxiv


In [3]:
from grobid_client.grobid_client import GrobidClient

client = GrobidClient(config_path='grobid_client_python/config.json')

GROBID server is up and running


In [ ]:
client.process_batch(
    input_path='pdfs',
    input_files = ['./pdfs/2010.11929v2.pdf', './pdfs/1706.03762v7.pdf', './pdfs/2305.02582v2.pdf'],
    output='output',
    n=10,
    generateIDs=False,
    consolidate_header=True,
    consolidate_citations=False,
    include_raw_citations=False,
    include_raw_affiliations=False,
    tei_coordinates=False,
    segment_sentences=False,
    force=True,
    verbose=False,
)

In [1]:
import chromadb
from langchain.vectorstores import chroma
from langchain_openai.embeddings import OpenAIEmbeddings
import os

COLLECTION_NAME = "arxiv"
PERSIST_DIR = "arxiv_vdb"
os.makedirs(PERSIST_DIR, exist_ok=True)
try:
    chromadb.PersistentClient('arxiv_vdb').delete_collection('arxiv')
except:
    pass
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vectordb = chroma.Chroma(
    persist_directory=PERSIST_DIR,
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings
)


In [2]:
vectordb

In [3]:
from arxiv_bot.search import IndexNewArxivPapers

IndexNewArxivPapers(
    vectordb,
    n_search_results=3,
    pdf_parser="PyMuPDF",
    chunk_size = 512,
    chunk_overlap=10
    
    )._run('transformers')

2024-02-13 20:10:19 - Loaded .env file
2024-02-13 20:10:19 - file_cache is only supported with oauth2client<4.0.0
2024-02-13 20:10:20 - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=&id_list=1706.03762%2C2402.02592%2C2010.11929&sortBy=relevance&sortOrder=descending&start=0&max_results=100
2024-02-13 20:10:21 - Got first page: 3 of 3 total results
GROBID server is up and running
2024-02-13 20:10:27 - Created a chunk of size 578, which is longer than the specified 512
2024-02-13 20:10:27 - Created a chunk of size 578, which is longer than the specified 512


/home/raihan/projects/arxiv-chatbot/venv/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


2024-02-13 20:10:29 - Created a chunk of size 537, which is longer than the specified 512
2024-02-13 20:10:29 - Created a chunk of size 551, which is longer than the specified 512
2024-02-13 20:10:29 - Created a chunk of size 1151, which is longer than the specified 512
2024-02-13 20:10:29 - Created a chunk of size 1253, which is longer than the specified 512
2024-02-13 20:10:29 - Created a chunk of size 537, which is longer than the specified 512
2024-02-13 20:10:29 - Created a chunk of size 551, which is longer than the specified 512
2024-02-13 20:10:29 - Created a chunk of size 1151, which is longer than the specified 512
2024-02-13 20:10:29 - Created a chunk of size 1253, which is longer than the specified 512
2024-02-13 20:10:30 - Created a chunk of size 544, which is longer than the specified 512
2024-02-13 20:10:30 - Created a chunk of size 606, which is longer than the specified 512
2024-02-13 20:10:30 - Created a chunk of size 534, which is longer than the specified 512
2024-0

In [5]:
chromadb.PersistentClient('arxiv_vdb').get_collection('arxiv').get()

{'ids': ['68b5f542-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f543-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f544-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f545-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f546-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f547-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f548-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f549-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f54a-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f54b-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f54c-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f54d-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f54e-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f54f-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f550-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f551-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f552-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f553-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f554-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f555-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f556-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f557-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f558-ca8a-11ee-bbf2-395025af1bc9',
  '68b5f559-ca8a-11ee-bbf2-

Get PDFs

In [64]:
client = arxiv.Client()
search = arxiv.Search(
    query="attention is all you need",
    max_results=5
)




In [65]:
res = client.results(search)

In [66]:
res = list(res)

In [168]:
import os
os.makedirs('./pdfs/', exist_ok=True)
os.makedirs('./source/', exist_ok=True)
for rs in res:
    rs.download_pdf('./pdfs/', filename=rs.entry_id.split('/')[-1]+'.pdf')
    # rs.download_source('./source/', filename=rs.entry_id.split('/')[-1]+'.tar.gz')

Naive RAG

In [ ]:
import chromadb
from typing import Literal, List
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.text_splitter import SpacyTextSplitter
import chainlit as cl
import time


In [1]:
from bs4 import BeautifulSoup #type: ignore
from dataclasses import dataclass
from dotenv import load_dotenv
from grobid_client.grobid_client import GrobidClient #type: ignore
from langchain.schema.document import Document
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.vectorstores import VectorStore
from typing import  Literal
import arxiv #type: ignore
import chainlit as cl
import chromadb
import os
import re
import time
from typing import List
import logging

logging.getLogger("IndexNewArxivPapers").setLevel(logging.INFO)
load_dotenv()


def read_tei(tei_file):
    with open(tei_file, 'r', encoding='utf-8') as tei:
        soup = BeautifulSoup(tei, 'lxml')
        return soup
    raise RuntimeError('Cannot generate a soup from the input')

def elem_to_text(elem, default=''):
    if elem:
        return elem.getText()
    else:
        return default

class TEIFile(object):
    def __init__(self, filename):
        self.filename = filename
        self.soup = read_tei(filename)
        self._text = None
        self._title = ''
        self._abstract = ''

    @property
    def doi(self):
        idno_elem = self.soup.find('idno', type='DOI')
        if not idno_elem:
            return ''
        else:
            return idno_elem.getText()

    @property
    def title(self):
        if not self._title:
            self._title = self.soup.title.getText() #type: ignore
        return self._title

    @property
    def abstract(self):
        if not self._abstract:
            abstract = self.soup.abstract.getText(separator=' ', strip=True) #type: ignore
            self._abstract = abstract
        return self._abstract

    @property
    def authors(self):
        authors_in_header = self.soup.analytic.find_all('author') #type: ignore

        result = []
        for author in authors_in_header:
            persname = author.persname
            if not persname:
                continue
            firstname = elem_to_text(persname.find("forename", type="first")).strip()
            middlename = elem_to_text(persname.find("forename", type="middle")).strip()
            surname = elem_to_text(persname.surname).strip()
            if middlename == '':
                full_name = f"{firstname} {surname}".strip()
            else:
                full_name = f"{firstname} {middlename} {surname}".strip()
            result.append(full_name)
        return result
    
    @property
    def text(self):
        if not self._text:
            divs_text = []
            for div in self.soup.body.find_all("div"): #type: ignore
                # div is neither an appendix nor references, just plain text.
                # if not div.get("type"):
                #     text = div.getText(separator=': ', strip=True).replace("\n", "")
                    
                #     divs_text.append(text)
                print(div)
            plain_text = "\n\n".join(divs_text)
            self._text = plain_text
        return self._text

2024-01-24 11:13:01 - Created default translation directory at /home/raihan/projects/arxiv-chatbot/.chainlit/translations
2024-01-24 11:13:01 - Created default translation file at /home/raihan/projects/arxiv-chatbot/.chainlit/translations/pt-BR.json
2024-01-24 11:13:01 - Created default translation file at /home/raihan/projects/arxiv-chatbot/.chainlit/translations/en-US.json


In [91]:
from grobid_client.grobid_client import GrobidClient
import os

client = GrobidClient(config_path="./grobid_client_python/config.json")

os.makedirs("./output/", exist_ok=True)
client.process(
    "processFulltextDocument",
    input_path = "./pdfs/",
    output="./output/",
    force=True,
    consolidate_citations=True,
    include_raw_citations=True,
    tei_coordinates=True,
    )



GROBID server is up and running


In [97]:
import json
import grobid_tei_xml
for file in os.listdir('./output/'):
    if file.endswith('.tei.xml'):
        xml_path = os.path.join('./output/', file)
        with open(xml_path, 'r') as xml_file:
            doc = grobid_tei_xml.parse_document_xml(xml_file.read())
                
        with open('./output/'+file+'.json', 'w') as json_file:
            json_file.write(json.dumps(doc.to_dict(), indent=2))
# xml_path = "./output/1912.11959v2.grobid.tei.xml"

# with open(xml_path, 'r') as xml_file:
#     doc = grobid_tei_xml.parse_document_xml(xml_file.read())

# with open('./output/2107.08000v1.grobid.tei.json', 'w') as json_file:
#     json_file.write(json.dumps(doc.to_dict(), indent=2))

In [95]:
print(doc.body)

Introduction The previous state-of-the-art sequence model, the recurrent neural network, has been largely supplanted by the Transformer model  [Vaswani et al., 2017] , which is primarily built atop a self-attention mechanism. Given a task to train upon, the self-attention mechanism focuses on one token per attention head within the entire sequence at each time-step; the key to the self-attention mechanism's success is the mechanism's ability to learn which token within the entire sequence to focus on in order to achieve the best results. The self-attention mechanism has proven successful on a variety of natural language processing tasks, but has not achieved ubiquitous success. The authors of  [Kaiser and Bengio, 2016]  pointed out that an attention mechanism would likely struggle to solve a task which required a model to focus on multiple tokens at a given time-step. Further, the authors of  [Kaiser and Sutskever, 2015]  recommended that an attention mechanism could be replaced by act

In [ ]:
with open("./output/2306.01926v1.grobid.tei.xml", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "lxml-xml")
chunks = []
content = ""
for div in soup.body.find_all(["div", "figure"]):
    if div.name == "figure":
        head = div.find("head")
        fig_desc = div.find("figDesc")
        # if ('Figure' not in head.getText()) and ('Table' not in head.getText()):
        content += head.getText().replace(" ", "") + fig_desc.getText() + "\n"
    elif div.name == "div":
        section_num = div.head.get("n")
        section_title = div.head.getText(separator=' ', strip=True)
        if section_num is None:
            section_num = ""
        content += section_num + " " + div.getText(separator=': ', strip=True) + "\n"
    content += "=======================================\n"
    # for head in div.find_all("head"):
    #     section_num = head.get("n")
    #     section_title = head.getText(separator=' ', strip=True)
    #     print(section_num, section_title)
    # content = ""
    
    # for p in div.find_all("p"):
    #     content += p.getText(separator=' ', strip=True)
    # print(content)
    # print("=================================")
    

Use Latex

In [12]:
#Unzip files in /source/ in a folder with the same name as the pdf

import tarfile

for file in os.listdir('./source/'):
    if file.endswith('.tar.gz'):
        tar = tarfile.open('./source/'+file, "r:gz")
        tar.extractall('./source/'+(file.replace('.tar.gz', '')))
        tar.close()
        os.remove('./source/'+file)

In [4]:
import os
from TexSoup import TexSoup

def read_and_parse_tex_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return parse_tex_content(content, os.path.dirname(file_path))

def parse_tex_content(content, base_dir):
    soup = TexSoup(content)
    full_content = str(soup)

    # Find all \input and \include commands and process them
    for command in soup.find_all('input') + soup.find_all('include'):
        filename = str(command.args[0]).replace('{', '').replace('}', '')
        if not filename.endswith('.tex'):
            filename += '.tex'
        
        included_file_path = os.path.join(base_dir, filename)
        if os.path.exists(included_file_path):
            included_content = read_and_parse_tex_file(included_file_path)
            full_content = full_content.replace(str(command), included_content)
        else:
            print(f"Warning: Included file {filename} not found.")

    return full_content

main_file_path = 'source/2107.08000v1/main.tex'
complete_document = read_and_parse_tex_file(main_file_path)

In [5]:
from TexSoup import TexSoup

soup = TexSoup(complete_document, tolerance=1)

In [6]:
soup.section

\section{Introduction}

In [7]:
print(complete_document)

with open('output.txt', 'w') as f:
    f.write(complete_document)

\documentclass[10pt,twocolumn,letterpaper]{article}

\usepackage{iccv}

%------------------------------------------------------------------------------
%------------------------------------------------------------------------------
% hack for CVPR/ICCV style
\makeatletter
\@namedef{ver@everyshi.sty}{}
\makeatother

%------------------------------------------------------------------------------
% main packages
\usepackage[dvipsnames,svgnames,x11names]{xcolor}
\usepackage{tikz}
\usetikzlibrary{arrows.meta,shapes,calc,matrix,fit,positioning,backgrounds,decorations.markings,fadings}
\usepackage{pgfplots}
\usepackage{pgfplotstable}
\pgfplotsset{compat=1.9}
\usepackage{xstring}

%------------------------------------------------------------------------------
% externalization: requires defining \finalcopy first

\usepgfplotslibrary{external}
% \tikzexternalize[prefix=fig/extern/]
\newcommand{\extfig}[2]{\tikzsetnextfilename{#1}{#2}}
\newcommand{\noextfig}[1]{\tikzset{external/export next={fal

In [10]:
import re

def extract_sections_from_string(tex_content):
    def remove_latex_commands(text):
        # Regular expression to match LaTeX commands
        func_regex = r'\\[a-zA-Z]+(\[[^\]]*\])?(\{[^\}]*\})?'
        return re.sub(func_regex, '', text)

    def remove_tables(text):
        # Remove content between \begin{table} and \end{table}
        regex = r'\\begin\{table\}.*?\\end\{table\}'
        return re.sub(regex, '', text, flags=re.DOTALL)

    def remove_comments(text):
        # Remove content between \begin{comment} and \end{comment}
        regex = r'%-*'
        return re.sub(regex, '', text)

    lines = tex_content.split('\n')

    sections = {}
    current_section = None
    section_content = []

    for line in lines:
        # Check for a new section, subsection, or paragraph
        match = re.match(r'\\(sub)*section{(.+?)}|\\paragraph{(.+?)}', line)
        if match:
            # Save the previous section
            if current_section is not None:
                content = ''.join(section_content).strip()
                content = remove_latex_commands(content)
                content = remove_tables(content)
                content = remove_comments(content)
                sections[current_section] = content
                section_content = []

            # Extract the new section name
            current_section = match.group(2) or match.group(3)
        elif current_section is not None:
            section_content.append(line)

    # Add the last section
    if current_section is not None:
        content = ''.join(section_content).strip()
        content = remove_latex_commands(content)
        content = remove_tables(content)
        content = remove_comments(content)
        sections[current_section] = content

    return sections

In [11]:
extract_sections_from_string(complete_document)

{'Introduction': ' (GLAM) involves both {channel} and {spatial} attention, as well as both {local} attention (channels/locations weighted independently, based on contextual information obtained by pooling) and {global} attention (based on pairwise interaction between channels/locations). As a result, four attention maps are used:  ($_c^l$),  ($_s^l$),  ($_c^g$) and  ($_s^g$). The input feature map $$ is weighted into local ($^l$) and global ($^g$) attention feature maps, which are fused with $$ to yield the  $^{gl}$. The diagram is abstract: The four attention modules are shown in more detail in Figures , , , .}Instance-level image retrieval is at the core of visual representation learning and is connected with many problems of visual recognition and machine learning, for instance ~, ~ and ~. Many large-scale open datasets~, and competitions have accelerated progress in instance-level image retrieval, which has been transformed by deep learning~.Many studies on instance-level image ret

PyMuPDF

In [33]:
import fitz
import os
for file in os.listdir('./pdfs/'):
    INTRO_DELIMITERS = ['Introduction\n', 'INTRODUCTION\n']
    REF_DELIMITERS = ['References\n','REFERENCES\n']
    
    intro_pattern = '|'.join(map(re.escape, INTRO_DELIMITERS))
    ref_delimiters = '|'.join(map(re.escape, REF_DELIMITERS))
    
    doc = fitz.open(f'./pdfs/{file}')
    id = file.replace('.pdf', '')
    content = ""
    for page in doc:
        content += page.get_text()
    text = re.split(intro_pattern, content)[-1]
    text = re.split(ref_delimiters, text)[0]


In [6]:
with open("output.txt", "w") as f:
    f.write(content)

In [21]:
import re

# Assuming 'delimiters' is a list of your delimiters

# Create a regular expression pattern that matches any of the delimiters

# Assuming 'content' is your string
sections = re.split(pattern, content, flags=re.IGNORECASE)

In [22]:
sections

['All the attention you need:\nGlobal-local, spatial-channel attention for image retrieval\nChull Hwan Song\nOdd Concepts\nHye Joo Han\nOdd Concepts\nYannis Avrithis\nInria, Univ Rennes, CNRS, IRISA\nAbstract\nWe address representation learning for large-scale\ninstance-level image retrieval. Apart from backbone, train-\ning pipelines and loss functions, popular approaches have\nfocused on different spatial pooling and attention mecha-\nnisms, which are at the core of learning a powerful global\nimage representation. There are different forms of attention\naccording to the interaction of elements of the feature tensor\n(local and global) and the dimensions where it is applied\n(spatial and channel). Unfortunately, each study addresses\nonly one or two forms of attention and applies it to different\nproblems like classiﬁcation, detection or retrieval.\nWe present global-local attention module (GLAM),\nwhich is attached at the end of a backbone network and\nincorporates all four forms of

In [2]:
content.split("Introduction")

['All the attention you need:\nGlobal-local, spatial-channel attention for image retrieval\nChull Hwan Song\nOdd Concepts\nHye Joo Han\nOdd Concepts\nYannis Avrithis\nInria, Univ Rennes, CNRS, IRISA\nAbstract\nWe address representation learning for large-scale\ninstance-level image retrieval. Apart from backbone, train-\ning pipelines and loss functions, popular approaches have\nfocused on different spatial pooling and attention mecha-\nnisms, which are at the core of learning a powerful global\nimage representation. There are different forms of attention\naccording to the interaction of elements of the feature tensor\n(local and global) and the dimensions where it is applied\n(spatial and channel). Unfortunately, each study addresses\nonly one or two forms of attention and applies it to different\nproblems like classiﬁcation, detection or retrieval.\nWe present global-local attention module (GLAM),\nwhich is attached at the end of a backbone network and\nincorporates all four forms of

Unstructured IO

In [72]:
from unstructured.partition.pdf import partition_pdf

doc = partition_pdf(
    filename = "pdfs/2107.08000v1.pdf",
    strategy = "hi_res",
    infer_table_structure=True,
    )

yolox_l0.05.onnx: 100%|██████████| 217M/217M [00:02<00:00, 98.1MB/s] 


2024-01-24 16:26:16 - Reading PDF for file: pdfs/2107.08000v1.pdf ...
2024-01-24 16:26:16 - Detecting page elements ...
2024-01-24 16:26:17 - Detecting page elements ...
2024-01-24 16:26:18 - Detecting page elements ...
2024-01-24 16:26:18 - Detecting page elements ...
2024-01-24 16:26:19 - Detecting page elements ...
2024-01-24 16:26:19 - Detecting page elements ...
2024-01-24 16:26:20 - Detecting page elements ...
2024-01-24 16:26:21 - Detecting page elements ...
2024-01-24 16:26:21 - Detecting page elements ...
2024-01-24 16:26:22 - Detecting page elements ...
2024-01-24 16:26:24 - Processing entire page OCR with tesseract...
2024-01-24 16:26:26 - Loading the Table agent ...
2024-01-24 16:26:26 - Loading the table structure model ...


config.json: 100%|██████████| 1.47k/1.47k [00:00<00:00, 9.55MB/s]
model.safetensors: 100%|██████████| 115M/115M [00:01<00:00, 72.4MB/s] 

2024-01-24 16:26:29 - Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)



model.safetensors: 100%|██████████| 46.8M/46.8M [00:00<00:00, 92.2MB/s]

2024-01-24 16:26:29 - [timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.



Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2024-01-24 16:26:31 - Processing entire page OCR with tesseract...
2024-01-24 16:26:33 - Processing entire page OCR with tesseract...
2024-01-24 16:26:33 - padding image by 20 for structure detection
2024-01-24 16:26:36 - Processing entire page OCR with tesseract...
2024-01-24 16:26:38 - Processing entire page OCR with tesseract...
2024-01-24 16:26:41 - Processing entire page OCR with tesseract...
2024-01-24 16:26:43 - Processing entire page OCR with tesseract...
2024-01-24 16:26:45 - Processing entire page OCR with tesseract...
2024-01-24 16:26:45 - padding image by 20 for structure detection
2024-01-24 16:26:45 - Processing entire page OCR with tesseract...
2024-01-24 16:26:46 - padding image by 20 for structure detection
2024-01-24 16:26:46 - Processing entire page OCR with tesseract...
2024-01-24 16:26:50 - Processing entire page OCR with tesseract...
2024-01-24 16:26:51 - padding image by 20 for structure detection
2024-01-24 16:26:52 - Processing entire page OCR with tesseract...

In [77]:
print(doc[0])

1 2 0 2


In [78]:
for el in doc:
    if 'Title' in repr(el):
        print(el.text)

All the attention you need: Global-local, spatial-channel attention for image retrieval
Chull Hwan Song Odd Concepts
Hye Joo Han Odd Concepts
Yannis Avrithis Inria, Univ Rennes, CNRS, IRISA
Abstract
Abstract
1. Introduction
1https://www.kaggle.com/c/landmark-retrieval-2020
local attention
2. Related work
3. Global-local attention
3.1. Local attention
F
Gc
3.2. Global attention
Gs
3.3. Global-local attention
4. Experiments
4.1. Datasets
4.2. Implementation details
4.3. Benchmarking
4.4. Ablation study
5. Conclusion
References


In [70]:
for el in doc:
    print(el.text)

1 2 0 2
l u J
6 1
]
V C . s c [ 1 v 0 0 0 8 0 . 7 0 1 2 : v i X r a
All the attention you need: Global-local, spatial-channel attention for image retrieval
Chull Hwan Song Odd Concepts
Hye Joo Han Odd Concepts
Yannis Avrithis Inria, Univ Rennes, CNRS, IRISA
Abstract
Abstract
We address representation learning for large-scale instance-level image retrieval. Apart from backbone, train- ing pipelines and loss functions, popular approaches have focused on different spatial pooling and attention mecha- nisms, which are at the core of learning a powerful global image representation. There are different forms of attention according to the interaction of elements of the feature tensor (local and global) and the dimensions where it is applied (spatial and channel). Unfortunately, each study addresses only one or two forms of attention and applies it to different problems like classiﬁcation, detection or retrieval.
We present global-local attention module (GLAM), which is attached at the end of 

In [86]:
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(content)

In [6]:
class IndexNewArxivPapers:
    """
    This tool indexes new papers from arxiv using the following steps:
    
    1. Get the paper ids from google search
    2. Download the papers
    3. Process the papers with GROBID
    4. Parse the TEI files
    5. Add the papers to the vectorstore
    
    """
    def __init__(self, 
                vectordb: VectorStore,
                n_search_results: int = 2,
                pdf_parser: Literal["grobid", "pymupdf"] = "grobid",
                chunk_size: int = 1024,
                chunk_overlap: int = 100,
                ):
    
        self.google_api = GoogleSearchAPIWrapper()
        self.arxiv_client = arxiv.Client(delay_seconds=0)
        self.vectordb = vectordb
        self.chromadb_client = chromadb.PersistentClient("arxiv_vdb").get_collection("arxiv")
        self.n_search_results = n_search_results
        self.splitter = SpacyTextSplitter(
            chunk_size = chunk_size,
            chunk_overlap = chunk_overlap,
            separator="\n\n")
        self.pdf_parser = pdf_parser
    
    def _get_paper_ids(self, query: str) -> List[str]:
        ARXIV_ID_REGEX =  r"\d{4}\.\d{4,5}"
        
        ids = list(
            {re.findall(ARXIV_ID_REGEX, result['link'])[0] 
             for result in self.google_api.results(query, self.n_search_results)}
            )
        
        return ids
    
    def _run(self, query):
        with cl.Step():
            self.ids = self._get_paper_ids(query)
            
            os.makedirs(f"./output/{query}", exist_ok=True)
            os.makedirs(f"./pdfs/{query}", exist_ok=True)
            
            for id in self.ids:
                if len(self.chromadb_client.get(where={'paper_id': id})['ids']) > 0:
                    self.ids.remove(id)
                else:
                    continue
            
            papers = list(self.arxiv_client.results(arxiv.Search(id_list=self.ids)))
            for paper in papers:
                id = paper.entry_id.split('/')[-1]
                paper.download_pdf(dirpath=f"./pdfs/{query}/", filename=f"{paper.entry_id.split('/')[-1]}.pdf")
            
            while not all([os.path.exists(f"./pdfs/{query}/{paper.entry_id.split('/')[-1]}.pdf") for paper in papers]):
                time.sleep(1)
            
            docs = []
            if self.pdf_parser == "grobid":
                self.grobid_client = GrobidClient(config_path="./grobid_client_python/config.json")
                self.grobid_client.process("processFulltextDocument", f"./pdfs/{query}/", output=f"./output/{query}", force=True)            
                
                while not all([os.path.exists(f"./output/{query}/{paper.entry_id.split('/')[-1]}.grobid.tei.xml") for paper in papers]):
                    time.sleep(1)
            
                for paper in papers:
                    tei_object = TEIFile(f"./output/{query}/{paper.entry_id.split('/')[-1]}.grobid.tei.xml")
                    chunks = self.splitter.split_text(tei_object.text)
                    docs.extend([
                            Document(
                                    page_content=chunk, 
                                    metadata={
                                        'paper_id': paper.entry_id.split("/")[-1],
                                        'authors': ", ".join([author.name for author in paper.authors]),
                                        'title': paper.title,
                                        'chunk_id': f"{paper.entry_id.split('/')[-1]}-{i}"
                                        }
                                    )
                                    for i, chunk in enumerate(chunks)
                                ]
                    )
                    
            if self.pdf_parser == "pymupdf":
                for paper in papers:
                    loader = PyMuPDFLoader(f"pdfs/{query}/{paper.entry_id.split('/')[-1]}.pdf")
                    chunks = loader.load_and_split(self.splitter)                    
                    docs.extend([
                            Document(
                                    page_content=chunk.page_content, 
                                    metadata={
                                        'paper_id': paper.entry_id.split("/")[-1],
                                        'authors': ", ".join([author.name for author in paper.authors]),
                                        'title': paper.title,
                                        'chunk_id': f"{paper.entry_id.split('/')[-1]}-{i}"
                                        }
                                    ) 
                                    for i, chunk in enumerate(chunks)
                                ]
                        )
            self.vectordb.add_documents(
                docs
            )


    async def _arun(self, query: str):
        async with cl.Step():
            self._run(query)

GROBID server is up and running


In [4]:
from arxiv_bot.search import TEIFile
from llama_index.vector_stores import ChromaVectorStore
from llama_index import Document, VectorStoreIndex
from langchain.text_splitter import SpacyTextSplitter

splitter = SpacyTextSplitter(chunk_size=1024, chunk_overlap=100, separator="\n\n")
for paper in os.listdir("./output/"):
    tei_object = TEIFile(f"./output/{paper}")
    print(tei_object.text)
    print("===================")



The key to a Transformer model is the self-attention mechanism, which allows the model to analyze an entire sequence in a computationally efficient manner. Recent work has suggested the possibility that general attention mechanisms used by RNNs could be replaced by active-memory mechanisms. In this work, we evaluate whether various activememory mechanisms could replace self-attention in a Transformer. Our experiments suggest that active-memory alone achieves comparable results to the self-attention mechanism for language modelling, but optimal results are mostly achieved by using both active-memory and self-attention mechanisms together. We also note that, for some specific algorithmic tasks, active-memory mechanisms alone outperform both the self attention and a combination of the two.

Introduction: The previous state-of-the-art sequence model, the recurrent neural network, has been largely supplanted by the Transformer model: [Vaswani et al., 2017]: , which is primarily built atop a

c:\Users\muraf\Courses\arxiv_llm\venv\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [64]:
docs.load_data()

[Document(id_='5890981b-8362-4a80-9cd6-7cf661effa8b', embedding=None, metadata={'page_label': '1', 'file_name': '1912.11959v2.pdf', 'file_path': 'pdfs\\1912.11959v2.pdf', 'file_type': 'application/pdf', 'file_size': 399865, 'creation_date': '2024-01-19', 'last_modified_date': '2024-01-19', 'last_accessed_date': '2024-01-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='a970d4d7e2cfc588d7ecbb820b81062b422baf9760e2fc006ef01d191ed6009d', text='Is Attention All What You Need ? - An Empirical Investigation on\nConvolution-Based Active Memory and Self-Attention\nThomas Dowdell ,Hongyu Zhang\nThe University of Newcastle, NSW, Australia\ntomjamesdowdell@gmail.com, hongyu.zhang@newcastle.edu.au\nAbstract\nThe key to a Transformer model is the self-attention\n

GROBID

In [70]:
import PyPDF2

reader = PyPDF2.PdfReader("./pdfs/2107.08000v1.pdf")
writer = PyPDF2.PdfWriter()
for i in range(len(reader.pages)):
    page = reader.pages[i]
    page.annotations.clear()
    writer.add_page(page)
    
    
with open('./pdfs/2107.08000v1.pdf', 'wb') as f:
    writer.write(f)
    
    

In [164]:
import fitz

doc = fitz.open("./pdfs/2107.08000v1.pdf")

doc.save("./pdfs/2107.08000v1_modified.pdf")

In [166]:
import fpdf


def convert_pdf_to_grayscale(input_pdf_path, output_pdf_path):
    # Open the PDF
    document = fitz.open(input_pdf_path)

    # Create a new PDF to store the grayscale pages
    grayscale_pdf = fitz.open()

    # Iterate through each page
    for page in document:
        links = page.get_links()
        for link in links:
            page.delete_link(link)
        # Convert the page to a pixmap (image) with colors converted to grayscale
        pixmap = page.get_pixmap(matrix=fitz.Matrix(2, 2), colorspace="GRAY")

        # Create a new blank page in the grayscale PDF with the same dimensions
        pdf_page = grayscale_pdf.new_page(width=pixmap.width, height=pixmap.height)

        # Insert the grayscale pixmap into the new page
        pdf_page.insert_image(pdf_page.rect, pixmap=pixmap)

    # Save the grayscale PDF
    grayscale_pdf.save(output_pdf_path)
    grayscale_pdf.close()
    document.close()

# Usage
convert_pdf_to_grayscale('./pdfs/2107.08000v1.pdf', './pdfs/2107.08000v1.pdf')

In [11]:
from grobid_client.grobid_client import GrobidClient

client = GrobidClient(config_path="./grobid_client_python/config.json")
client.process(
    "processFulltextDocument",
    "./pdfs/",
    output="./output/",
    force=True,
    n=20
    )



GROBID server is up and running


In [41]:
from bs4 import BeautifulSoup

with open("./output/2107.08000v1.grobid.tei.xml", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "lxml-xml")
chunks = []
content = ""
for div in soup.body.find_all(["div", "figure"]):
    if div.name == "figure":
        head = div.find("head")
        fig_desc = div.find("figDesc")
        # if ('Figure' not in head.getText()) and ('Table' not in head.getText()):
        if( 'Figure' not in head.get_text()) and ('Table' not in head.get_text()):
            content += head.getText().replace(" ", "") + fig_desc.getText() + "\n"
    elif div.name == "div":
        section_num = div.head.get("n")
        section_title = div.head.getText(separator=' ', strip=True)
        if section_num is None:
            section_num = ""
        content += section_num + " " + div.getText(separator= ' ', strip=True) + "\n"
    content += "=======================================\n"

In [42]:
print(content)

1. Introduction Instance-level image retrieval is at the core of visual representation learning and is connected with many problems of visual recognition and machine learning, for instance metric learning [30, 26] , few-shot learning [42] and unsupervised learning [8] . Many large-scale open datasets [3, 37, 16, 29, 53] , and competitions 1 have accelerated progress in instance-level image retrieval, which has been transformed by deep learning [3] . Many studies on instance-level image retrieval focus on learning features from convolutional neural networks (CNN), while others focus on re-ranking, for instance by graph-based methods [11] . The former can be distinguished according to feature types: local descriptors, reminiscent of SIFT [27] , where an image is mapped to a few hundred vectors; and global descriptors, where an image is mapped to a 1 https://www.kaggle.com/c/landmark-retrieval-2020 single vector. In fact, deep learning has brought global descriptors with astounding perfor